In [1]:
#==== PyTOUGH ==== 
#    geometry
from mulgrids import *
#================= 

# calls to system
import os

from numpy import *

In [2]:
#en esta variable guardamos la dirección del directorio de trabajo
cwd=os.getcwd()
cwd

'/Users/fer/Desktop/tallerPyTOUGH/4_MulgridConGMSH_Elevs_Voronoi'

In [3]:
#carga las elevaciones del área de estudio dada una malla Delaunay de GMSH
#el archivo elevations_Delaunay.txt se construye con la ayuda de QGis para obtener las elevaciones
elevs=np.loadtxt('elevations_Voronoi.txt')
#elevs

In [4]:
#cuál es la mínima elevación en el área
min(elevs[:,2])

216.0

In [5]:
#crea un objeto mulgrid a partir de una malla GMSH
geo=mulgrid().from_gmsh('miMallaGMSH.msh', [250.0]*14, convention=0, atmos_type=1, top_elevation=min(elevs[:,2]), chars = ascii_lowercase, spaces=True, block_order=None)
geo

267 nodes; 476 columns; 15 layers; 7140 blocks; 0 wells
Naming convention: 0 (3 characters for column, 2 digits for layer)
Atmosphere type  : 1 (one atmosphere block over each column)

In [6]:
# Este código crea un archivo 'in' para que AMESH construya una malla VORONOI
f=open(cwd+'/amesh/in','w')
f.write('locat\n')

tol=0.1
thck=250.
z0=min(elevs[:,2])
zc=z0+thck*0.5
i=1
for ly in range(1,15):
    zc=zc-thck
    for node in geo.nodelist:
        xi=node.pos[0]
        yi=node.pos[1]
        f.write(format(i,"5d")+format(ly,"5d")+format(xi,"20.10e")+format(yi,"20.10e")+format(zc,"20.10e")+format(thck,"20.10e")+'\n')
        i=i+1
f.write('\n')

f.write('toler\n')
f.write(format(tol,"10.4f")+'\n')
f.write('\n')

#corners of the polygon - clockwise
f.write('bound\n')
f.write(format(0.,'9.3f')+' '+format(1000.,'9.3f')+'\n')
f.write(format(0.,'9.3f')+' '+format(15000.,'9.3f')+'\n')
f.write(format(2000.,'9.3f')+' '+format(16000.,'9.3f')+'\n')
f.write(format(10000.,'9.3f')+' '+format(16000.,'9.3f')+'\n')
f.write(format(12000.,'9.3f')+' '+format(15000.,'9.3f')+'\n')
f.write(format(12000.,'9.3f')+' '+format(1000.,'9.3f')+'\n')
f.write(format(10000.,'9.3f')+' '+format(0.,'9.3f')+'\n')
f.write(format(2000.,'9.3f')+' '+format(0.,'9.3f')+'\n')

f.close()


#ATENCIÓN: este comando ejecuta AMESH en mac - revisa cómo ejecurarlo en tu sistema
os.system('cd amesh;./amesh6_osx')

0

In [7]:
#Importamos la malla a PyTOUGH
#use convention=2 for definition of GENER SL,NS - NS given by column name
#0 - 3 characters for column followed by 2 digits for layer
#1 - 3 characters for layer followed by 2 digits for column
#2 - 2 characters for layer followed by 3 digits for column
ameshgeo=mulgrid().from_amesh(input_filename=cwd+'/amesh/in', segment_filename=cwd+'/amesh/segmt', convention=0, node_tolerance=None, justify='r', chars=ascii_lowercase, spaces=True, block_order =None)
geo=ameshgeo[0]
geo.atmosphere_type=1

In [10]:
geo

540 nodes; 267 columns; 15 layers; 4005 blocks; 0 wells
Naming convention: 0 (3 characters for column, 2 digits for layer)
Atmosphere type  : 1 (one atmosphere block over each column)

In [13]:
geo

540 nodes; 267 columns; 15 layers; 4005 blocks; 0 wells
Naming convention: 0 (3 characters for column, 2 digits for layer)
Atmosphere type  : 1 (one atmosphere block over each column)

In [14]:
#Esta opción genera error
#geo.fit_surface(elevs, alpha=0.1, beta=0.1, columns=[], min_columns=[], grid_boundary=False, layer_snap=0.0, silent=False)

#Otra forma de generar la elevación
i=0 
for cl in geo.columnlist[:]:
    cl.surface=elevs[i,2]
    i=i+1

In [15]:
geo.write_vtk(filename='4geometry.vtu', arrays=None, wells=False, blockmap={}, surface_snap =0.1)